# SD-IPADAPTER

In [ ]:
# --- SD3.5-LARGE + IP-Adapter (Img2Img) ---
!pip -q install -U "diffusers>=0.33.0" "transformers>=4.43.0" accelerate safetensors pillow opencv-python bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import os, glob, gc, contextlib, torch
from PIL import Image
from tqdm import tqdm
import cv2
from itertools import islice

from diffusers import StableDiffusion3Img2ImgPipeline
from transformers import SiglipVisionModel, SiglipImageProcessor

# Bellek/performans
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64,expandable_segments:True"
torch.set_float32_matmul_precision("high")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [ ]:
# --- Model/Adapter IDs ---
MODEL_ID = "stabilityai/stable-diffusion-3.5-large"
IPADAPTER_ID = "InstantX/SD3.5-Large-IP-Adapter"
IPADAPTER_REV = "f1f54ca369ae759f9278ae9c87d46def9f133c78"
IMAGE_ENCODER_ID = "google/siglip-so400m-patch14-384"


DROP_T5 = True

#--- Prompts ---
BASE_PROMPT = (
    "photorealistic small boat on the sea, natural lighting, sharp hull edges,"
    "realistic water ripples and reflections"
)
NEG_PROMPT = "lowres, blurry, cartoon, duplicate boats, text, watermark, logo, deformed hull"

# BASE_PROMPT = (
#     "photorealistic wide-angle view of a busy sea with multiple large cargo ships and oil tankers in the distance, small patrol boats in the foreground, calm ocean waves with realistic water ripples and reflections,"
#     "partly cloudy sky, natural daylight, sharp and detailed ship structures, cinematic composition"
# )
# NEG_PROMPT = "lowres, blurry, cartoon, overexposed, underexposed, extra ships, duplicate boats, text, watermark, logo, deformed ships, unrealistic water, distorted horizon, over-saturated colors"

# --- Parametreler --- # bunların üstünde değişiklik yapabiliriz.
GUIDANCE = 4.3
STEPS = 24              # 20–28
STRENGTH = 0.28         # 0.18–0.35: küçük kaynaklara sadakat için düşük
LONG_EDGE = 768      #768    # 512–768 küçük kaynaklar için güvenli
IP_SCALE = 0.70         # 0.5–0.9 arası kompozisyon kilidi
USE_EMBEDS = True       # True: ip_adapter_image_embeds; False: ip_adapter_image
VARIANTS_PER_IMAGE = 1 #1
BASE_SEED = 2025

# --- IO ---
CANDIDATE_DIRS = ["/content/drive/MyDrive/boat_dataset/roboflow_onlyboat"]
INPUT_DIR = next((d for d in CANDIDATE_DIRS if os.path.isdir(d)), None)
assert INPUT_DIR, f"Girdi klasörü bulunamadı. Şunlardan biri olmalı: {CANDIDATE_DIRS}"

OUT_DIR = "/content/boat_out_sd35_i2i_ip"
os.makedirs(OUT_DIR, exist_ok=True)

max_images = None


In [ ]:
def to_mult16(n: int) -> int:
    return int(round(max(n, 16) / 16) * 16)

def list_images(folder: str):
    exts = (".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff")
    files = []
    for e in exts:
        files += glob.glob(os.path.join(folder, f"*{e}"))
    return sorted(files)

def pre_upscale_min(img_pil: Image.Image, min_long_edge: int = 96) -> Image.Image:
    """
    21x8 gibi aşırı küçük görselleri önce küçük bir eşik üzerine çıkar.
    Interpolasyon: Bicubic (OpenCV). Yumuşatma yok, detay kaybını azaltmak için direkt ölçek.
    """
    w, h = img_pil.size
    m = max(w, h)
    if m >= min_long_edge:
        return img_pil

    scale = float(min_long_edge) / float(m)
    new_w, new_h = max(16, int(w * scale)), max(16, int(h * scale))
    im = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    up = cv2.resize(im, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    up = cv2.cvtColor(up, cv2.COLOR_BGR2RGB)
    return Image.fromarray(up)

def resize_long_edge(img: Image.Image, long_edge: int = LONG_EDGE) -> Image.Image:
    w, h = img.size
    s = long_edge / max(w, h)
    new_w, new_h = to_mult16(int(w * s)), to_mult16(int(h * s))
    return img.resize((new_w, new_h), Image.LANCZOS)


In [ ]:

feature_extractor = SiglipImageProcessor.from_pretrained(IMAGE_ENCODER_ID)
image_encoder = SiglipVisionModel.from_pretrained(
    IMAGE_ENCODER_ID, torch_dtype=torch.float16
)

pipe_kwargs = dict(
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    low_cpu_mem_usage=True,
)


if DROP_T5:
    pipe_kwargs.update(text_encoder_3=None, tokenizer_3=None)

print("Model yükleniyor:", MODEL_ID)
pipe = StableDiffusion3Img2ImgPipeline.from_pretrained(
    MODEL_ID,
    feature_extractor=feature_extractor,  # IP-Adapter için processor
    image_encoder=image_encoder,          # SigLIP encoder modülü
    **pipe_kwargs,
)


if DROP_T5:
    if hasattr(pipe, "text_encoder_3"): pipe.text_encoder_3 = None
    if hasattr(pipe, "tokenizer_3"):    pipe.tokenizer_3 = None

#Safety checker'ı kapat (hafifletme)
if hasattr(pipe, "safety_checker"): pipe.safety_checker = None
if hasattr(pipe, "requires_safety_checker"): pipe.requires_safety_checker = False

#
pipe.load_ip_adapter(
    IPADAPTER_ID,
    weight_name="ip-adapter.bin",
    revision=IPADAPTER_REV,
)
pipe.set_ip_adapter_scale(IP_SCALE)

# VAE / attention optimizasyonları
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()
try:
    pipe.enable_sdpa()  # torch 2.x
except Exception:
    pipe.enable_attention_slicing()


if device.type == "cuda":

    pipe.enable_model_cpu_offload()
else:
    pipe.to("cpu")

progress bar sessiz
try:
    pipe.set_progress_bar_config(disable=True)
except Exception:
    pass


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

Model yükleniyor: stabilityai/stable-diffusion-3.5-large


model_index.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ip-adapter.bin:   0%|          | 0.00/1.60G [00:00<?, ?B/s]

In [ ]:
def print_devices(pipe):
    try: print("unet     :", next(pipe.unet.parameters()).device)
    except: pass
    try: print("vae      :", next(pipe.vae.parameters()).device)
    except: pass
    try: print("te1      :", next(pipe.text_encoder.parameters()).device)
    except: pass
    try: print("te2      :", next(pipe.text_encoder_2.parameters()).device)
    except: pass
    try: print("img_enc  :", next(pipe.image_encoder.parameters()).device)
    except: pass
    try:
        # IP-Adapter modülü adı sürüme göre değişebiliyor
        mod = getattr(pipe, "ip_adapter", None) or getattr(pipe, "image_proj_model", None)
        if mod is not None:
            print("ip_adapter:", next(mod.parameters()).device)
    except:
        pass

print_devices(pipe)


vae      : cpu
te1      : cpu
te2      : cpu
img_enc  : cpu


In [ ]:
import numpy as np
all_imgs = list_images(INPUT_DIR)
assert all_imgs, f"Girdi klasöründe görsel yok: {INPUT_DIR}"

iterable = all_imgs if max_images is None else islice(all_imgs, max_images)
total_to_process = len(all_imgs) if max_images is None else min(len(all_imgs), int(max_images))

print(f"Bulunan görsel: {len(all_imgs)}; İşlenecek: {total_to_process}")
print(f"Çıkış klasörü: {OUT_DIR}")

processed = 0
skipped = 0

# Autocast bağlamı
autocast_ctx = (
    torch.autocast(device_type="cuda", dtype=torch.float16)
    if device.type == "cuda" else contextlib.nullcontext()
)

for idx, path in enumerate(tqdm(iterable, total=total_to_process, desc="Processing")):
    try:
        img = Image.open(path).convert("RGB")
    except Exception as e:
        print("Skip (okuma hatası):", path, e)
        skipped += 1
        continue

    # Ultra küçükler için min pre-upscale + hedef uzun kenara ölçek
    img_up = pre_upscale_min(img, min_long_edge=96)
    up = resize_long_edge(img_up, LONG_EDGE)
    #up = resize_long_edge(img, LONG_EDGE)
    # Deterministik üretim için seed
    g = torch.Generator(device=device).manual_seed(BASE_SEED + idx)


    if USE_EMBEDS:
        ip_embeds = pipe.prepare_ip_adapter_image_embeds(
            ip_adapter_image=up,
            device=device,
            num_images_per_prompt=1,
            do_classifier_free_guidance=(GUIDANCE > 1.0),
        )

    for k in range(VARIANTS_PER_IMAGE):
        with autocast_ctx, torch.inference_mode():
            kwargs = dict(
                prompt=BASE_PROMPT,
                negative_prompt=NEG_PROMPT,
                image=up,
                strength=STRENGTH,
                guidance_scale=GUIDANCE,
                num_inference_steps=STEPS,
                generator=g,
            )
            if USE_EMBEDS:
                kwargs["ip_adapter_image_embeds"] = ip_embeds
            else:
                kwargs["ip_adapter_image"] = up

            result = pipe(**kwargs)

        out_img = result.images[0]
        name = os.path.splitext(os.path.basename(path))[0]
        suffix = f"_sd35IP_L{LONG_EDGE}_st{STRENGTH}_ip{IP_SCALE}_gs{GUIDANCE}_s{STEPS}"
        if VARIANTS_PER_IMAGE > 1:
            suffix += f"_v{k+1}"
        out_path = os.path.join(OUT_DIR, f"{name}{suffix}.png")
        out_img.save(out_path)

        del out_img, result

    processed += 1
    del up, img_up, img
    if device.type == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

print(f"Bitti. İşlenen: {processed}, Atlanan: {skipped}, Çıktı klasörü: {OUT_DIR}")


Bulunan görsel: 51; İşlenecek: 51
Çıkış klasörü: /content/boat_out_sd35_i2i_ip


Processing: 100%|██████████| 51/51 [29:36<00:00, 34.84s/it]

Bitti. İşlenen: 51, Atlanan: 0, Çıktı klasörü: /content/boat_out_sd35_i2i_ip


In [ ]:
!zip -r /content/BOAT.zip /content/boat_out_sd35_i2i_ip


  adding: content/boat_out_sd35_i2i_ip/ (stored 0%)
  adding: content/boat_out_sd35_i2i_ip/MVI_0790_VIS_OB_frame250_jpg.rf.cd382286fafded9b6d0e0fbc51a754bb_Boat_0_sd35IP_L768_st0.28_ip0.7_gs4.3_s24.png (deflated 0%)
  adding: content/boat_out_sd35_i2i_ip/MVI_0790_VIS_OB_frame210_jpg.rf.57a67503f2e0c4deace7a12b44bf4bba_Boat_0_sd35IP_L768_st0.28_ip0.7_gs4.3_s24.png (deflated 0%)
  adding: content/boat_out_sd35_i2i_ip/MVI_1584_VIS_frame410_jpg.rf.81c3b286e0f76bee9f4a5362e3015b20_Boat_1_sd35IP_L768_st0.28_ip0.7_gs4.3_s24.png (deflated 0%)
  adding: content/boat_out_sd35_i2i_ip/MVI_1523_NIR_frame530_jpg.rf.67775de1f35a6cceac8a53dafc10a1a1_Boat_0_sd35IP_L768_st0.28_ip0.7_gs4.3_s24.png (deflated 0%)
  adding: content/boat_out_sd35_i2i_ip/MVI_1523_NIR_frame520_jpg.rf.59b899843843e97704fdc0a2ab2f76e1_Boat_0_sd35IP_L768_st0.28_ip0.7_gs4.3_s24.png (deflated 0%)
  adding: content/boat_out_sd35_i2i_ip/MVI_0801_VIS_OB_frame355_jpg.rf.525a71d1b00e23d46f252c9c8e40b10b_Boat_0_sd35IP_L768_st0.28_ip0.7_g

In [ ]:
# Kompozisyona daha çok sadakat (daha “aynı” kompozisyon)
#STRENGTH = 0.20; IP_SCALE = 0.85; GUIDANCE = 3.8

# Promptu biraz öne çıkar (stilden emin değilken)
# STRENGTH = 0.30; IP_SCALE = 0.60; GUIDANCE = 5.0

# Aşırı küçük kaynak (<=32 px uzun kenar) için:
# LONG_EDGE = 512; STRENGTH = 0.22; IP_SCALE = 0.80; STEPS = 26
